<a href="https://pt.wikipedia.org/wiki/S%C3%A3o_Paulo"><img src = "https://upload.wikimedia.org/wikipedia/commons/thumb/3/36/Panorama_de_S%C3%A3o_Paulo.jpg/1920px-Panorama_de_S%C3%A3o_Paulo.jpg" width = 900  height = 500> </a>

# Pet Shop Analysys in 2 Neighborhoods in São Paulo

**São Paulo**  is a municipality in the Southeast Region of Brazil. The metropolis is an alpha global city (as listed by the GaWC) and the most populous city in Brazil, the Western Hemisphere and the Southern Hemisphere, besides being the largest Portuguese-speaking city in the world. The municipality is also the world's 12th largest city proper by population. The city is the capital of the surrounding state of São Paulo, the most populous and wealthiest state in Brazil. 
More and more the population of São Paulo is buying/adopting Pets, and also Pet Shops to take care of them. 
At this analysis I'd like to compare 2 Neighborhoods: Campo Belo and the most populous neighborhood

# Data Aquisition

### Population

For this Project I'll use population of each neighborhood in Sâo Paulo. 
I found on Wikipedia a Dataset containing the population of each neighborhood in São Paulo, but need some preparation for this study:  
- There are 2 Population Datas of each Neighborhood that need to be studied;  
- All Columns are in Portuguese;  
- Find the Campo Belo and the Most populous Neighborhood data.

### Coords
Using Geopy library, find the Latitude and Longitude of each place studied.

###  Foursquare API
Find the Pet Shops / Clinics and plot them

In [1]:
# Importing the libraries for this project
import numpy as np # library to handle data in a vectorized manne
import pandas as pd # library for data analsysis
import requests # library to handle requests
from pandas.io.json import json_normalize # tranforming json file into a pandas dataframe library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library

print("OK, Libraries Imported")


OK, Libraries Imported


Getting the data of Populating per Neighborhood from **Wikipedia**

In [2]:
url = pd.read_html("https://pt.wikipedia.org/wiki/Lista_dos_distritos_de_S%C3%A3o_Paulo_por_popula%C3%A7%C3%A3o", attrs={"class": "wikitable"})

In [3]:
neigh_pop= url[0]
neigh_pop.head()

,Posição,Distrito,População 2010,Censo 2000
0,1,Grajaú,444.593,333.436
1,2,Sapopemba,296.042,282.239
2,3,Jardim Ângela,291.798,245.805
3,4,Brasilândia,280.069,247.328
4,5,Capão Redondo,275.230,240.793


In [4]:
neigh_pop.info() # Checking if are there any NaN values
neigh_pop.shape # Checking the size of the Dataframe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 4 columns):
Posição           96 non-null int64
Distrito          96 non-null object
População 2010    96 non-null float64
Censo 2000        96 non-null float64
dtypes: float64(2), int64(1), object(1)
memory usage: 3.1+ KB


(96, 4)

### There are 2 Datas indicating Population here I will consider the **População 2010**  otherwise the data will be from 2000

In [5]:
neigh_pop.drop(columns = "Censo 2000", inplace = True )
neigh_pop.head()

,Posição,Distrito,População 2010
0,1,Grajaú,444.593
1,2,Sapopemba,296.042
2,3,Jardim Ângela,291.798
3,4,Brasilândia,280.069
4,5,Capão Redondo,275.230


In [6]:
# Renaming all the columns to english
neigh_pop.rename(columns = {"Posição":"Place" , "Distrito":"Neighborhood", "População 2010":"Population" } , inplace = True)
neigh_pop

,Place,Neighborhood,Population
0,1,Grajaú,444.593
1,2,Sapopemba,296.042
2,3,Jardim Ângela,291.798
3,4,Brasilândia,280.069
4,5,Capão Redondo,275.230
...,...,...,...
91,92,Jaguara,24.114
92,93,Sé,21.564
93,94,Pari,16.144
94,95,Barra Funda,12.977


In [7]:
neigh_pop.shape

(96, 3)

### Lets Find the population of Campo Belo and The 1st most populous in this Dataset

In [8]:
pcb = neigh_pop[neigh_pop["Neighborhood"].str.match("Campo Belo")]
pcb

,Place,Neighborhood,Population
71,72,Campo Belo,62.53


In [9]:
fisrt = neigh_pop.head(1)
fisrt

,Place,Neighborhood,Population
0,1,Grajaú,444.593


In [10]:
calc = 444.593 / 62.530
calc

7.1100751639213176

 Grajaú is the most Populous place in São Paulo 7 times more than Campo Belo

### Getting coordenates of each Neighborhood

In [11]:
address = 'Campo Belo, SP'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Campo Belo SP are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Campo Belo SP are -23.626730549999998, -46.669421429340844.


In [12]:
# At this point the geolocator was not working, so I got the Data from Google Maps
latitude_GR = -23.772121 
longitude_GR = -46.668243
print('The geograpical coordinate of Grajau SP are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Grajau SP are -23.626730549999998, -46.669421429340844.


### Define Foursquare Credentials and Version

In [13]:
CLIENT_ID = "YZRNT2IDJGZBMBQMMU5VUVVDNQLWFG502ZZBTCZSVK0MN5HD" # your Foursquare ID
CLIENT_SECRET = "XX4WAKH50MF203ROAPGM1GXXZZLVBVRM43H2N15QGMQM212O" # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YZRNT2IDJGZBMBQMMU5VUVVDNQLWFG502ZZBTCZSVK0MN5HD
CLIENT_SECRET:XX4WAKH50MF203ROAPGM1GXXZZLVBVRM43H2N15QGMQM212O


In [14]:
search_query = 'Pet'
radius = 750
print(search_query + ' .... OK!')

Pet .... OK!


In [15]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=YZRNT2IDJGZBMBQMMU5VUVVDNQLWFG502ZZBTCZSVK0MN5HD&client_secret=XX4WAKH50MF203ROAPGM1GXXZZLVBVRM43H2N15QGMQM212O&ll=-23.626730549999998,-46.669421429340844&v=20180604&query=Pet&radius=750&limit=30'

In [16]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e8def11bae9a2001bcca012'},
 'response': {'venues': [{'id': '52a60b7f11d282bb061d38ce',
    'name': 'Pet Shop O Mundo de Bono',
    'location': {'address': 'R. João Álvares Soares',
     'crossStreet': 'Av. Vieira de Moraes',
     'lat': -23.62398488133133,
     'lng': -46.67395824314344,
     'labeledLatLngs': [{'label': 'display',
       'lat': -23.62398488133133,
       'lng': -46.67395824314344}],
     'distance': 554,
     'postalCode': '04609-002',
     'cc': 'BR',
     'city': 'São Paulo',
     'state': 'SP',
     'country': 'Brasil',
     'formattedAddress': ['R. João Álvares Soares (Av. Vieira de Moraes)',
      'São Paulo, SP',
      '04609-002',
      'Brasil']},
    'categories': [{'id': '4bf58dd8d48988d100951735',
      'name': 'Pet Store',
      'pluralName': 'Pet Stores',
      'shortName': 'Pet Store',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/pet_store_',
       'suffix': '.png'},
      'primary': True}]

In [17]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
cbpets = json_normalize(venues)
cbpets.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,52a60b7f11d282bb061d38ce,Pet Shop O Mundo de Bono,"[{'id': '4bf58dd8d48988d100951735', 'name': 'P...",v-1586360410,False,R. João Álvares Soares,Av. Vieira de Moraes,-23.623985,-46.673958,"[{'label': 'display', 'lat': -23.6239848813313...",554,04609-002,BR,São Paulo,SP,Brasil,[R. João Álvares Soares (Av. Vieira de Moraes)...
1,5dc1a14716293d00082469f6,Euro Pet,"[{'id': '4bf58dd8d48988d100951735', 'name': 'P...",v-1586360410,False,677 Rua Estêvão Baião,NaN,-23.626581,-46.667913,"[{'label': 'display', 'lat': -23.626581, 'lng'...",154,04624-002,BR,São Paulo,SP,Brasil,"[677 Rua Estêvão Baião, São Paulo, SP, 04624-0..."
2,589b649c7220e630f2044937,Pet Shop - Centro Médico Veterinário Robert Koch,"[{'id': '4bf58dd8d48988d104941735', 'name': 'M...",v-1586360410,False,"R. Cristóvão Pereira, 1693",Rua Vieira de Moraes,-23.624730,-46.668510,"[{'label': 'display', 'lat': -23.62473, 'lng':...",241,04617-015,BR,São Paulo,SP,Brasil,"[R. Cristóvão Pereira, 1693 (Rua Vieira de Mor..."
3,4f14406fe4b0456269ec238e,Pet Club,"[{'id': '4bf58dd8d48988d100951735', 'name': 'P...",v-1586360410,False,Rua Edson 1266,NaN,-23.625177,-46.671597,"[{'label': 'display', 'lat': -23.6251774605318...",281,NaN,BR,SP,SP,Brasil,"[Rua Edson 1266, SP, SP, Brasil]"
4,51601d5fe4b0b5e5f123accf,Pet In The City,"[{'id': '5032897c91d4c4b30a586d69', 'name': 'P...",v-1586360410,False,NaN,NaN,-23.623811,-46.669240,"[{'label': 'display', 'lat': -23.6238110410969...",325,NaN,BR,NaN,NaN,Brasil,[Brasil]


In [18]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in cbpets.columns if col.startswith('location.')] + ['id']
dataframe_filtered = cbpets.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Pet Shop O Mundo de Bono,Pet Store,R. João Álvares Soares,Av. Vieira de Moraes,-23.623985,-46.673958,"[{'label': 'display', 'lat': -23.6239848813313...",554,04609-002,BR,São Paulo,SP,Brasil,[R. João Álvares Soares (Av. Vieira de Moraes)...,52a60b7f11d282bb061d38ce
1,Euro Pet,Pet Store,677 Rua Estêvão Baião,NaN,-23.626581,-46.667913,"[{'label': 'display', 'lat': -23.626581, 'lng'...",154,04624-002,BR,São Paulo,SP,Brasil,"[677 Rua Estêvão Baião, São Paulo, SP, 04624-0...",5dc1a14716293d00082469f6
2,Pet Shop - Centro Médico Veterinário Robert Koch,Medical Center,"R. Cristóvão Pereira, 1693",Rua Vieira de Moraes,-23.624730,-46.668510,"[{'label': 'display', 'lat': -23.62473, 'lng':...",241,04617-015,BR,São Paulo,SP,Brasil,"[R. Cristóvão Pereira, 1693 (Rua Vieira de Mor...",589b649c7220e630f2044937
3,Pet Club,Pet Store,Rua Edson 1266,NaN,-23.625177,-46.671597,"[{'label': 'display', 'lat': -23.6251774605318...",281,NaN,BR,SP,SP,Brasil,"[Rua Edson 1266, SP, SP, Brasil]",4f14406fe4b0456269ec238e
4,Pet In The City,Pet Service,NaN,NaN,-23.623811,-46.669240,"[{'label': 'display', 'lat': -23.6238110410969...",325,NaN,BR,NaN,NaN,Brasil,[Brasil],51601d5fe4b0b5e5f123accf
5,Pet Club Clínica Veterinária,Pet Store,Rua Edson 1266,NaN,-23.625821,-46.672362,"[{'label': 'display', 'lat': -23.625821, 'lng'...",316,NaN,BR,SP,SP,Brasil,"[Rua Edson 1266, SP, SP, Brasil]",5214dee911d2f8576d63d10e
6,Adriano Ozonio Pet,Pet Store,"República do Iraque,1403",NaN,-23.623150,-46.669490,"[{'label': 'display', 'lat': -23.6231500838270...",398,NaN,BR,São Paulo,SP,Brasil,"[República do Iraque,1403, São Paulo, SP, Brasil]",4d8b8d9776e1236adf0a3243
7,Pet Shop Fofocão,Pet Store,NaN,NaN,-23.624222,-46.672528,"[{'label': 'display', 'lat': -23.624222, 'lng'...",422,NaN,BR,NaN,NaN,Brasil,[Brasil],4fae7aeae4b01d7a900d04a3
8,Pet Play,Pet Store,"Rua Demostenes,1054",NaN,-23.619223,-46.670327,"[{'label': 'display', 'lat': -23.6192229624314...",840,NaN,BR,São Paulo,SP,Brasil,"[Rua Demostenes,1054, São Paulo, SP, Brasil]",4eda7004e5fa78b16e357423
9,Pet Beau Champs,Veterinarian,NaN,NaN,-23.624297,-46.665811,"[{'label': 'display', 'lat': -23.6242974215085...",457,NaN,BR,NaN,NaN,Brasil,[Brasil],4ef4d47649010be35eb42d04


There are 3 places that s not Pet Centers in the results, lets drop this datas from de Dataframe

In [19]:
dataframe_filtered.drop([19,21,22], inplace = True )
dataframe_filtered.reset_index(inplace = True)
dataframe_filtered.drop(columns = ["index"], inplace = True)
dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Pet Shop O Mundo de Bono,Pet Store,R. João Álvares Soares,Av. Vieira de Moraes,-23.623985,-46.673958,"[{'label': 'display', 'lat': -23.6239848813313...",554,04609-002,BR,São Paulo,SP,Brasil,[R. João Álvares Soares (Av. Vieira de Moraes)...,52a60b7f11d282bb061d38ce
1,Euro Pet,Pet Store,677 Rua Estêvão Baião,NaN,-23.626581,-46.667913,"[{'label': 'display', 'lat': -23.626581, 'lng'...",154,04624-002,BR,São Paulo,SP,Brasil,"[677 Rua Estêvão Baião, São Paulo, SP, 04624-0...",5dc1a14716293d00082469f6
2,Pet Shop - Centro Médico Veterinário Robert Koch,Medical Center,"R. Cristóvão Pereira, 1693",Rua Vieira de Moraes,-23.624730,-46.668510,"[{'label': 'display', 'lat': -23.62473, 'lng':...",241,04617-015,BR,São Paulo,SP,Brasil,"[R. Cristóvão Pereira, 1693 (Rua Vieira de Mor...",589b649c7220e630f2044937
3,Pet Club,Pet Store,Rua Edson 1266,NaN,-23.625177,-46.671597,"[{'label': 'display', 'lat': -23.6251774605318...",281,NaN,BR,SP,SP,Brasil,"[Rua Edson 1266, SP, SP, Brasil]",4f14406fe4b0456269ec238e
4,Pet In The City,Pet Service,NaN,NaN,-23.623811,-46.669240,"[{'label': 'display', 'lat': -23.6238110410969...",325,NaN,BR,NaN,NaN,Brasil,[Brasil],51601d5fe4b0b5e5f123accf
5,Pet Club Clínica Veterinária,Pet Store,Rua Edson 1266,NaN,-23.625821,-46.672362,"[{'label': 'display', 'lat': -23.625821, 'lng'...",316,NaN,BR,SP,SP,Brasil,"[Rua Edson 1266, SP, SP, Brasil]",5214dee911d2f8576d63d10e
6,Adriano Ozonio Pet,Pet Store,"República do Iraque,1403",NaN,-23.623150,-46.669490,"[{'label': 'display', 'lat': -23.6231500838270...",398,NaN,BR,São Paulo,SP,Brasil,"[República do Iraque,1403, São Paulo, SP, Brasil]",4d8b8d9776e1236adf0a3243
7,Pet Shop Fofocão,Pet Store,NaN,NaN,-23.624222,-46.672528,"[{'label': 'display', 'lat': -23.624222, 'lng'...",422,NaN,BR,NaN,NaN,Brasil,[Brasil],4fae7aeae4b01d7a900d04a3
8,Pet Play,Pet Store,"Rua Demostenes,1054",NaN,-23.619223,-46.670327,"[{'label': 'display', 'lat': -23.6192229624314...",840,NaN,BR,São Paulo,SP,Brasil,"[Rua Demostenes,1054, São Paulo, SP, Brasil]",4eda7004e5fa78b16e357423
9,Pet Beau Champs,Veterinarian,NaN,NaN,-23.624297,-46.665811,"[{'label': 'display', 'lat': -23.6242974215085...",457,NaN,BR,NaN,NaN,Brasil,[Brasil],4ef4d47649010be35eb42d04


###  São Paulo City Map

<img src = "http://www.mapas-sp.com/imagens/bairros-sp.jpg" width = 300  height = 200>

In [20]:
address = 'São Paulo, SP'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude_sp = location.latitude
longitude_sp = location.longitude
print('The geograpical coordinate of São Paulo SP are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of São Paulo SP are -23.626730549999998, -46.669421429340844.


In [21]:
sp_map = folium.Map(location=[latitude_sp, longitude_sp], zoom_start=9) # generate map centred around the Conrad Hotel

folium.features.CircleMarker(
        [latitude, longitude],
        radius=5,
        color='blue',
        #popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(sp_map)

folium.features.CircleMarker(
        [latitude_GR, longitude_GR],
        radius=5,
        color='blue',
        #popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(sp_map)

# display map
sp_map

Both Neighborhood are in South of São Paulo City

### Now using the Data from **FourSquare** API, lets Plot the cooordenatesof each Pet Store/ Clinic

In [22]:
cb_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around the Conrad Hotel
# add the Petshops as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(cb_map)

# display map
cb_map

In [23]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_GR, longitude_GR, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=YZRNT2IDJGZBMBQMMU5VUVVDNQLWFG502ZZBTCZSVK0MN5HD&client_secret=XX4WAKH50MF203ROAPGM1GXXZZLVBVRM43H2N15QGMQM212O&ll=-23.772121,-46.668243&v=20180604&query=Pet&radius=750&limit=30'

In [24]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e8df0171e152c001b4291b7'},
 'response': {'venues': []}}

 As I suspect there are no Pet Shop or Clinics in Grajaú, at least in Foursquare Database

In [25]:
grajau_map = folium.Map(location=[latitude_GR, longitude_GR], zoom_start=16) # generate map centred around the Conrad Hotel
grajau_map

## Conclusion

There are a lot of Pet Shops / Pet Clinics in Campo Belo/SP, almost 1 stores per 2 square. If you compare to Grajaú there are no Pet Shops or clinics.  
This results could have bias, because it depend of the person to insert the data of Pet Shops, but could represent a lot of aspects including the variety of stores in each neighborhood.  
At the other point of view, could be a nice place to open a Petshop and use the social medias like Foursquare because there are not Pet Center in Grajaú.  
For more informations consult the Final Report